In [1]:
%pip install numpy pandas scikit-learn xgboost lightgbm requests tqdm joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install nba_api pandas scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [12]:
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
import pandas as pd
games = LeagueGameFinder(
    season_nullable="2024-25",
    league_id_nullable="00",
    season_type_nullable="Regular Season"
).get_data_frames()[0]

games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['MIN'] > 0]
games = games[~games['MATCHUP'].str.contains('All-Star', na=False)]
games


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612754,IND,Indiana Pacers,0022401189,2025-04-13,IND @ CLE,W,289,126,...,0.909,14,47,61,27,15,10,13,20,8.0
1,22024,1610612740,NOP,New Orleans Pelicans,0022401196,2025-04-13,NOP vs. OKC,L,240,100,...,0.767,11,36,47,22,6,4,12,19,-15.0
2,22024,1610612738,BOS,Boston Celtics,0022401187,2025-04-13,BOS vs. CHA,W,241,93,...,0.692,15,34,49,22,12,6,13,14,7.0
3,22024,1610612746,LAC,LA Clippers,0022401198,2025-04-13,LAC @ GSW,W,265,124,...,0.778,9,33,42,28,11,3,16,21,5.0
4,22024,1610612753,ORL,Orlando Magic,0022401186,2025-04-13,ORL @ ATL,L,240,105,...,0.667,12,31,43,25,11,5,14,8,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22024,1610612740,NOP,New Orleans Pelicans,0022400069,2024-10-23,NOP vs. CHI,W,241,123,...,0.905,7,35,42,29,15,10,11,17,12.0
2456,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
2457,22024,1610612750,MIN,Minnesota Timberwolves,0022400062,2024-10-22,MIN @ LAL,L,239,103,...,0.741,12,35,47,17,4,1,15,22,-7.0
2458,22024,1610612752,NYK,New York Knicks,0022400061,2024-10-22,NYK @ BOS,L,241,109,...,0.750,5,29,34,20,2,3,11,12,-23.0


In [10]:
from nba_api.stats.static import teams
from pprint import pprint

all_teams = teams.get_teams()
pprint(all_teams, width=200)

[{'abbreviation': 'ATL', 'city': 'Atlanta', 'full_name': 'Atlanta Hawks', 'id': 1610612737, 'nickname': 'Hawks', 'state': 'Georgia', 'year_founded': 1949},
 {'abbreviation': 'BOS', 'city': 'Boston', 'full_name': 'Boston Celtics', 'id': 1610612738, 'nickname': 'Celtics', 'state': 'Massachusetts', 'year_founded': 1946},
 {'abbreviation': 'CLE', 'city': 'Cleveland', 'full_name': 'Cleveland Cavaliers', 'id': 1610612739, 'nickname': 'Cavaliers', 'state': 'Ohio', 'year_founded': 1970},
 {'abbreviation': 'NOP', 'city': 'New Orleans', 'full_name': 'New Orleans Pelicans', 'id': 1610612740, 'nickname': 'Pelicans', 'state': 'Louisiana', 'year_founded': 2002},
 {'abbreviation': 'CHI', 'city': 'Chicago', 'full_name': 'Chicago Bulls', 'id': 1610612741, 'nickname': 'Bulls', 'state': 'Illinois', 'year_founded': 1966},
 {'abbreviation': 'DAL', 'city': 'Dallas', 'full_name': 'Dallas Mavericks', 'id': 1610612742, 'nickname': 'Mavericks', 'state': 'Texas', 'year_founded': 1980},
 {'abbreviation': 'DEN', '

In [47]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['MIN'] > 0]
games = games[~games['MATCHUP'].str.contains('All-Star', na=False)]

playoff_teams = [
    # West
    1610612760,  # OKC Thunder
    1610612745,  # HOU Rockets
    1610612747,  # LAL Lakers
    1610612743,  # DEN Nuggets
    1610612746,  # LAC Clippers
    1610612750,  # MIN Timberwolves
    1610612744,  # GSW Warriors
    1610612763,  # MEM Grizzlies

    # East
    1610612739,  # CLE Cavaliers
    1610612738,  # BOS Celtics
    1610612752,  # NYK Knicks
    1610612754,  # IND Pacers
    1610612749,  # MIL Bucks
    1610612765,  # DET Pistons
    1610612753,  # ORL Magic
    1610612748   # MIA Heat
]

games = games[games['TEAM_ID'].isin(playoff_teams)]

games['a_home'] = games['MATCHUP'].str.contains(r' vs\.').astype(int)

a = games.rename(columns=lambda c: 'a_'+c if c not in ['GAME_ID','GAME_DATE'] else c)
b = games.rename(columns=lambda c: 'b_'+c if c not in ['GAME_ID','GAME_DATE'] else c)

merged = (
    a.merge(b, on=['GAME_ID','GAME_DATE'])
     .query('a_TEAM_ID != b_TEAM_ID')
)
merged['a_win'] = (merged['a_WL'] == 'W').astype(int)

merged['a_efg'] = (merged['a_FGM'] + 0.5 * merged['a_FG3M']) / merged['a_FGA']
merged['b_efg'] = (merged['b_FGM'] + 0.5 * merged['b_FG3M']) / merged['b_FGA']
merged['a_ts']  = merged['a_PTS'] / (2 * merged['a_FGA'] + 0.44 * merged['a_FTA'])
merged['b_ts']  = merged['b_PTS'] / (2 * merged['b_FGA'] + 0.44 * merged['b_FTA'])

# 2) 30-game rolling means of efg and ts
for stat in ['efg','ts']:
    merged[f'a_{stat}_30gm'] = (
        merged.groupby('a_TEAM_ID')[f'a_{stat}']
              .rolling(30, min_periods=1).mean()
              .reset_index(level=0, drop=True)
    )
    merged[f'b_{stat}_30gm'] = (
        merged.groupby('b_TEAM_ID')[f'b_{stat}']
              .rolling(30, min_periods=1).mean()
              .reset_index(level=0, drop=True)
    )
    merged[f'diff_{stat}_30gm'] = merged[f'a_{stat}_30gm'] - merged[f'b_{stat}_30gm']

# 3) Rebounding & assist/turnover differentials (season‐to‐date)
merged['a_rim'] = merged['a_OREB'] + merged['a_DREB']
merged['b_rim'] = merged['b_OREB'] + merged['b_DREB']
merged['a_ast_to'] = merged['a_AST'] / merged['a_TOV'].replace(0,1)
merged['b_ast_to'] = merged['b_AST'] / merged['b_TOV'].replace(0,1)

for stat in ['rim','ast_to']:
    merged[f'a_{stat}_30gm'] = (
        merged.groupby('a_TEAM_ID')[f'a_{stat}']
              .rolling(30, min_periods=1).mean()
              .reset_index(level=0, drop=True)
    )
    merged[f'b_{stat}_30gm'] = (
        merged.groupby('b_TEAM_ID')[f'b_{stat}']
              .rolling(30, min_periods=1).mean()
              .reset_index(level=0, drop=True)
    )
    merged[f'diff_{stat}_30gm'] = merged[f'a_{stat}_30gm'] - merged[f'b_{stat}_30gm']

# 4) Rest‐day differential
merged['a_rest'] = (merged.groupby('a_TEAM_ID')['GAME_DATE']
                          .diff().dt.days.fillna(1))
merged['b_rest'] = (merged.groupby('b_TEAM_ID')['GAME_DATE']
                          .diff().dt.days.fillna(1))
merged['diff_rest'] = merged['a_rest'] - merged['b_rest']

# 5) Head‑to‑head win% so far this season
h2h = (merged.groupby(['a_TEAM_ID','b_TEAM_ID'])['a_win']
           .expanding().mean()
           .reset_index()
           .rename(columns={'a_win':'h2h_pct'}))
merged = merged.merge(
    h2h[['a_TEAM_ID','b_TEAM_ID','h2h_pct']],
    on=['a_TEAM_ID','b_TEAM_ID'],
    how='left'
)
merged['h2h_pct'].fillna(0.5, inplace=True)

# 6) Assemble the enhanced feature matrix with updated column names
feature_cols = [
    'a_a_home',
    'diff_efg_30gm', 'diff_ts_30gm',
    'diff_rim_30gm', 'diff_ast_to_30gm',
    'diff_rest',   'h2h_pct'
]
X = merged[feature_cols]
y = merged['a_win']


C:\Users\ducta\AppData\Local\Temp\ipykernel_36484\1110041009.py:95: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['h2h_pct'].fillna(0.5, inplace=True)


In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)
model = LogisticRegression().fit(X_train, y_train)
preds = model.predict_proba(X_val)[:,1]
print("AUC =", roc_auc_score(y_val, preds))

AUC = 0.7473443223443224


In [49]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
clf = LogisticRegression(max_iter=1000)
grid = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print("best C:", grid.best_params_['C'])
print("cv AUC:", grid.best_score_)
model = grid.best_estimator_

best C: 100
cv AUC: 0.8031646485088952


In [50]:
# 1. Lock in your best‐C model and check hold‑out AUC
model = grid.best_estimator_
val_preds = model.predict_proba(X_val)[:,1]
print("Hold‑out AUC =", roc_auc_score(y_val, val_preds))

# 2. (Optional but recommended) Calibrate if you see mis‑calibration
from sklearn.calibration import CalibratedClassifierCV
calib = CalibratedClassifierCV(model, method='isotonic', cv=5)
calib.fit(X_train, y_train)
val_preds_cal = calib.predict_proba(X_val)[:,1]
print("Calibrated AUC =", roc_auc_score(y_val, val_preds_cal))

# 3. Compute a best‑of‑7 series win‑probability function
from scipy.stats import binom

def series_win_prob(p):
    # P(win ≥4 out of 7) = ∑_{k=4}^7 C(7,k) p^k (1−p)^(7−k)
    return sum(binom.pmf(k, 7, p) for k in range(4, 8))

# 4. Example: get per‑game p for Team A vs B, then series p
#    (assuming you've built a feature row `x_feat` for A @ home first)
# Here we define a sample x_feat using the input features used in the model.
x_feat = merged[feature_cols].iloc[-1].values
p_game = model.predict_proba([x_feat])[0,1]   # or calib.predict_proba
p_series = series_win_prob(p_game)
print(f"P(game) = {p_game:.3f}, P(series) = {p_series:.3f}")


Hold‑out AUC = 0.7389194139194138
Calibrated AUC = 0.7376373626373627
P(game) = 0.548, P(series) = 0.604


In [51]:
# Given playoff_teams = [West seeds 1–8, then East seeds 1–8]
west = playoff_teams[:8]
east = playoff_teams[8:]

# Round 1: 1 vs 8, 2 vs 7, 3 vs 6, 4 vs 5 in each conference
first_round = [
    # East matchups
    (east[0], east[7]),  # 1 vs 8
    (east[1], east[6]),  # 2 vs 7
    (east[2], east[5]),  # 3 vs 6
    (east[3], east[4]),  # 4 vs 5

    # West matchups
    (west[0], west[7]),  # 1 vs 8
    (west[1], west[6]),  # 2 vs 7
    (west[2], west[5]),  # 3 vs 6
    (west[3], west[4]),  # 4 vs 5
]

In [52]:
print("Playoffs:", playoff_teams)

Playoffs: [1610612760, 1610612745, 1610612747, 1610612743, 1610612746, 1610612750, 1610612744, 1610612763, 1610612739, 1610612738, 1610612752, 1610612754, 1610612749, 1610612765, 1610612753, 1610612748]


In [53]:
raw_games = leaguegamefinder.LeagueGameFinder(
    season_nullable="2024-25",
    league_id_nullable="00"
).get_data_frames()[0]

# 1) Drop DNPs
df = raw_games[raw_games['MIN'] > 0].copy()

# 2) Flag home/away
df['home'] = df['MATCHUP'].str.contains(r' vs\.').astype(int)

# 3) Merge in opponent points
opp = df[['GAME_ID','PTS']].rename(columns={'PTS':'opp_PTS'})
df = df.merge(opp, on='GAME_ID')

# 4) Per‑game diff & 5‑game rolling mean
df['diff'] = df['PTS'] - df['opp_PTS']
df = df.sort_values(['TEAM_ID','GAME_DATE'])
df['diff_30gm'] = (
    df
    .groupby('TEAM_ID')['diff']
    .rolling(window=30, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# 5) Final rolling diff for every team
last_diff = df.groupby('TEAM_ID')['diff_30gm'].last().to_dict()

In [54]:
from scipy.stats import binom

def series_win_prob(p):
    return sum(binom.pmf(k, 7, p) for k in range(4, 8))

last_diff = merged.groupby('a_TEAM_ID')['diff_ts_30gm'].last().to_dict()

series_p = {}
for A, B in first_round:
    d = last_diff[A] - last_diff[B]
    # Construct feature vector with dummy/default values for the missing features.
    # Our feature_cols are:
    # [a_a_home, diff_efg_10gm, diff_ts_10gm, diff_rim_10gm, diff_ast_to_10gm, diff_rest, h2h_pct]
    # We'll assume a_a_home=1, diff_efg_10gm=0, diff_ts_10gm=d, diff_rim_10gm=0, diff_ast_to_10gm=0,
    # diff_rest=0, and h2h_pct=0.5 as default values.
    features = [1, 0, d, 0, 0, 0, 0.5]
    p_game = model.predict_proba([features])[0, 1]
    series_p[(A, B)] = series_win_prob(p_game)

In [55]:
import warnings

# ignore only the "X does not have valid feature names" message
warnings.filterwarnings(
    action='ignore',
    message='X does not have valid feature names',
    category=UserWarning
)

In [58]:
import random
from collections import Counter

# helper to simulate one best‑of‑7
def simulate_series(A, B):
    if (A, B) in series_p:
        prob = series_p[(A, B)]
    else:
        # fallback: compute series win probability using last_diff and model
        d = last_diff[A] - last_diff[B]
        # Construct a feature vector with dummy/default values similar to cell index 12.
        features = [1, 0, d, 0, 0, 0, 0.5]
        p_game = model.predict_proba([features])[0,1]
        prob = series_win_prob(p_game)
    return A if random.random() < prob else B

# simulate an entire bracket
def simulate_bracket():
    # Round 1: eight series in first_round order
    r1 = [simulate_series(A, B) for A,B in first_round]

    # Round 2: East winners vs each other, then West
    east_sf = [simulate_series(r1[0], r1[3]), simulate_series(r1[1], r1[2])]
    west_sf = [simulate_series(r1[4], r1[7]), simulate_series(r1[5], r1[6])]

    # Conference Finals
    east_f = simulate_series(east_sf[0], east_sf[1])
    west_f = simulate_series(west_sf[0], west_sf[1])

    # NBA Finals
    return simulate_series(east_f, west_f)

counts_r2 = Counter()       # winners of Round 1 → make Round 2 (Conf. Semis)
counts_r3 = Counter()       # winners of Round 2 → make Round 3 (Conf. Finals)
counts_finals = Counter()   # winners of Round 3 → make NBA Finals
counts_champs = Counter()   # champions

N = 20000
for _ in range(N):
    # Round 1: simulate eight series (16 teams → 8 winners)
    r1 = [simulate_series(A, B) for A, B in first_round]
    counts_r2.update(r1)

    # Round 2: separate conference semis
    east_sf = [simulate_series(r1[i], r1[i+1]) for i in (0, 2)]
    west_sf = [simulate_series(r1[i], r1[i+1]) for i in (4, 6)]
    r2 = east_sf + west_sf
    counts_r3.update(r2)

    # Round 3: Conference Finals
    east_f = simulate_series(east_sf[0], east_sf[1])
    west_f = simulate_series(west_sf[0], west_sf[1])
    finals = [east_f, west_f]
    counts_finals.update(finals)

    # Championship
    champ = simulate_series(east_f, west_f)
    counts_champs[champ] += 1

# Build probability DataFrame
probs = []
for tid in playoff_teams:
    probs.append({
        'team_id':            tid,
        'make_R2 (8)':        counts_r2[tid] / N,
        'make_R3 (4)':        counts_r3[tid] / N,
        'make_Finals (2)':    counts_finals[tid] / N,
        'champion (1)':       counts_champs[tid] / N
    })

df_probs = pd.DataFrame(probs)
try:
    id_to_name
except NameError:
    from nba_api.stats.static import teams
    all_teams = teams.get_teams()
    id_to_name = {team['id']: team['full_name'] for team in all_teams}

df_probs['team_name'] = df_probs['team_id'].map(id_to_name)
df_probs = df_probs[['team_name','make_R2 (8)','make_R3 (4)','make_Finals (2)','champion (1)']]

print(df_probs.sort_values('champion (1)', ascending=False))

# Explanation:
# The simulation starts from the first round defined in cell 10, which consists of 8 matchups (16 teams).
# Only one team per matchup advances, so only the 8 winners from round 1 (and any teams that advance further)
# can become champions. That is why the final counters (champs) include only around 8 teams.

                 team_name  make_R2 (8)  make_R3 (4)  make_Finals (2)  \
8      Cleveland Cavaliers      0.63505      0.50090          0.38120   
15              Miami Heat      0.36495      0.29720          0.23460   
7        Memphis Grizzlies      0.52200      0.42500          0.30100   
9           Boston Celtics      0.79730      0.17780          0.11845   
11          Indiana Pacers      0.74520      0.32840          0.10305   
10         New York Knicks      0.73440      0.44675          0.10435   
2       Los Angeles Lakers      0.76865      0.51085          0.20475   
3           Denver Nuggets      0.86960      0.33905          0.14545   
0    Oklahoma City Thunder      0.47800      0.31485          0.16355   
6    Golden State Warriors      0.40250      0.11980          0.06955   
13         Detroit Pistons      0.26560      0.13680          0.02460   
12         Milwaukee Bucks      0.25480      0.08805          0.02185   
1          Houston Rockets      0.59750      0.1403